In [48]:
## Run when initialise the code


import pandas as pd
from pandas import DataFrame
import geopandas as gpd
from shapely.ops import nearest_points

from geopandas import GeoDataFrame
from shapely.geometry import Point, MultiPolygon,LineString, MultiLineString
from shapely.ops import linemerge, unary_union,split
from shapely.strtree import STRtree
from shapely.geometry.base import BaseGeometry
from shapely.prepared import prep

import os
import requests
from tqdm import tqdm
from sklearn.cluster import DBSCAN

import numpy as np
import math
from math import log2

from itertools import groupby,combinations
from collections import defaultdict

import warnings
warnings.filterwarnings(action='ignore')

pjr_loc = os.path.dirname(os.getcwd())
project_crs = 'epsg:3857'
place = 'Turin,Italy'
place_folder = f'{pjr_loc}/places'
os.makedirs(place_folder , exist_ok=True)
data_folder = f'{place_folder}/{place.replace(",","_")}'
os.makedirs(data_folder, exist_ok=True)
test_folder = f'{data_folder}/test'
os.makedirs(test_folder, exist_ok=True)

In [209]:

def get_bbox_from_nominatim(place_name):
    """
    Fetches the bounding box of a place using the Nominatim API.

    Parameters:
        place_name (str): The name of the place to search.

    Returns:
        tuple: (south, north, west, east) as floats.
    """
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": place_name,
        "format": "json",
        "limit": 1
    }
    headers = {
        "User-Agent": "GeoResearcher/1.0 (achituv@ariel.ac.il)"  # Update if needed
    }

    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    results = response.json()

    if not results:
        raise ValueError(f"No results found for place: {place_name}")

    bbox = results[0]["boundingbox"]
    return float(bbox[0]), float(bbox[1]), float(bbox[2]), float(bbox[3])

def download_osm_roads_bbox(place_name, highway_tags=None):
    """
    Downloads OpenStreetMap road data within the bounding box of a given place name.

    Parameters:
    ----------
    place_name : str
        Name of the place to retrieve the road data for. The function uses the Nominatim API
        to geocode this name into a bounding box.

    highway_tags : list of str, optional
        A list of OSM 'highway' tag values to filter the types of roads to download.
        If not provided, a default list including common road and path types is used:
        ['primary','footway','pedestrian','cycleway','path','secondary',
         'tertiary','unclassified','residential','service','living_street','steps']

    Returns:
    -------
    GeoDataFrame
        A GeoDataFrame containing the downloaded road features within the bounding box.
        Each feature includes its geometry (LineString), 'osmid' (OSM way ID), and other OSM tags.
        CRS is EPSG:4326.
    """
    if highway_tags is None:
        highway_tags = [
            'primary', 'footway', 'pedestrian', 'cycleway', 'path',
            'secondary', 'tertiary', 'unclassified', 'residential',
            'service', 'living_street', 'steps'
        ]

    # Get bounding box from place name
    south, north, west, east = get_bbox_from_nominatim(place_name)
    highway_filter = '|'.join(highway_tags)

    # Construct Overpass QL query
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json][timeout:180];
    (
      way["highway"~"{highway_filter}"]({south},{west},{north},{east});
    );
    out geom;
    """

    # Send query to Overpass API
    response = requests.get(overpass_url, params={'data': query})
    response.raise_for_status()
    data = response.json()

    # Convert results to GeoDataFrame
    roads = []
    for element in data['elements']:
        if element['type'] == 'way' and 'geometry' in element:
            coords = [(p['lon'], p['lat']) for p in element['geometry']]
            if len(coords) >= 2:
                geom = LineString(coords)
                tags = element.get('tags', {})
                tags['geometry'] = geom
                tags['osmid'] = element['id']
                roads.append(tags)

    gdf = gpd.GeoDataFrame(roads, geometry='geometry', crs='EPSG:4326')
    return gdf


def remove_unconnected_streets(df_connected, str_name='name_left', con_str_name='name_right'):
    """
    Remove streets from a GeoDataFrame that are not connected to any other streets.

    Parameters:
    df_connected (GeoDataFrame): The input GeoDataFrame containing street data.
    str_name (str): The column name for the street name in the first dataset.
    con_str_name (str): The column name for the connected street name in the second dataset.

    Returns:
    GeoDataFrame: A new GeoDataFrame with unconnected streets removed.
    """
    df_analysis = df_connected.copy()
    s_join_analysis = gpd.sjoin(df_analysis, df_connected)
    s_join_analysis2 = s_join_analysis[s_join_analysis[str_name] != s_join_analysis[con_str_name]]
    not_connected = set(df_connected['name']) - set(s_join_analysis2.reset_index()[str_name])
    df_pro_filtered = df_connected[~df_connected['name'].isin(not_connected)]
    return df_pro_filtered

def calculate_bearing(line):
    """
    Calculate the bearing (azimuth) in degrees from the start to the end of a LineString.

    Parameters:
    -----------
    line : shapely.geometry.LineString
        The input LineString geometry.

    Returns:
    --------
    float
        Bearing in degrees (0° to 360°), or NaN if the geometry is invalid or too short.
    """
    if not line or line.is_empty or len(line.coords) < 2:
        return np.nan

    x1, y1 = line.coords[0]
    x2, y2 = line.coords[-1]
    angle = np.degrees(np.arctan2(x2 - x1, y2 - y1))
    return (angle + 360) % 360


def merge_lines_by_attributes(gdf, attrs=['name', 'highway']):
    """
    Efficiently merges line geometries in a GeoDataFrame by grouping them based on specified attributes.

    Parameters:
        gdf (GeoDataFrame): Input GeoDataFrame with line geometries.
        attrs (list of str): List of attribute names to group by.

    Returns:
        GeoDataFrame: New GeoDataFrame with merged LineStrings or MultiLineStrings split into parts.
    """
    records = []
    grouped = gdf.groupby(attrs, sort=False)

    for key, group in tqdm(grouped, total=len(grouped), desc="Merging lines"):
        lines = list(group.geometry)

        if len(lines) == 1:
            records.append({**dict(zip(attrs, key)), 'geometry': lines[0]})
            continue

        merged = linemerge(MultiLineString(lines))

        if isinstance(merged, LineString):
            records.append({**dict(zip(attrs, key)), 'geometry': merged})
        elif isinstance(merged, MultiLineString):
            for part in merged.geoms:
                records.append({**dict(zip(attrs, key)), 'geometry': part})

    return gpd.GeoDataFrame(records, crs=gdf.crs)


def extract_internal_intersections_spatial_index(geoms):
    """
    Extracts internal intersection points between geometries using spatial indexing (STRtree).

    Parameters:
        geoms (list of BaseGeometry): List of Shapely geometries (e.g., LineStrings).

    Returns:
        list of Point: Intersection points where geometries overlap internally.
    """
    geoms = [g for g in geoms if isinstance(g, BaseGeometry) and not g.is_empty]
    tree = STRtree(geoms)
    all_points = set()

    for idx, geom in enumerate(tqdm(geoms, total=len(geoms), desc="Extracting intersections")):
        candidate_idxs = tree.query(geom)
        candidates = [geoms[i] for i in candidate_idxs]
        geom =geom.buffer(0.001)
        for other in candidates:
            if other is geom or not geom.intersects(other):
                continue

            inter = geom.intersection(other)
            if inter.is_empty:
                continue

            if inter.geom_type == 'Point':
                all_points.add(inter)
            elif inter.geom_type=='LineString':
                all_points.add(Point(inter.coords[0]))
            elif inter.geom_type.startswith('Multi') or inter.geom_type == 'GeometryCollection':
                for g in inter.geoms:
                    if g.geom_type == 'Point':
                        all_points.add(g)
                    elif g.geom_type == 'LineString':
                        all_points.add(Point(g.coords[0]))
        # Round each point to 3 decimal places
    rounded = [Point(round(pt.x, 3), round(pt.y, 3)) for pt in list(all_points)]

    # Remove duplicates by converting to a set of coordinates
    unique_coords = set((pt.x, pt.y) for pt in rounded)

    # Convert back to list of Point objects
    return [Point(x, y) for x, y in unique_coords]


def split_lines_by_intersections_fast(gdf, intersections, buffer_eps=1e-6):
    """
    Splits line geometries at intersection points using a buffered boundary union.

    Parameters:
        gdf (GeoDataFrame): Input GeoDataFrame containing line geometries.
        intersections (list of Point): Points where lines should be split.
        buffer_eps (float): Buffer radius to create boundaries around each intersection point for splitting.

    Returns:
        GeoDataFrame: New GeoDataFrame with split line segments, preserving 'name' and 'highway' attributes.
    """
    if not intersections:
        return gdf.copy()

    # Build splitting geometry
    splitters = unary_union([pt.buffer(buffer_eps).boundary for pt in intersections])
    prepared_splitters = prep(splitters)

    records = []
    for row in gdf.itertuples(index=False):
        geom = row.geometry

        if prepared_splitters.intersects(geom):
            try:
                split_parts = split(geom, splitters)
                for part in split_parts.geoms:
                    if part.length > 0:
                        records.append({
                            'name': getattr(row, 'name', None),
                            'highway': getattr(row, 'highway', None),
                            'geometry': part
                        })
            except Exception:
                # Fallback in case of split failure
                records.append({
                    'name': getattr(row, 'name', None),
                    'highway': getattr(row, 'highway', None),
                    'geometry': geom
                })
        else:
            records.append({
                'name': getattr(row, 'name', None),
                'highway': getattr(row, 'highway', None),
                'geometry': geom
            })

    temp_geo = gpd.GeoDataFrame(records, crs=gdf.crs)
    gdf = temp_geo[temp_geo.length >1e-4]
    gdf['geometry'] = gdf['geometry'].apply(
    lambda geom: LineString([(round(x, 3), round(y, 3)) for x, y in geom.coords])
)
    return gdf


In [6]:
gdf_roads = download_osm_roads_bbox(place)[['highway','name','osmid','junction','geometry']].to_crs(project_crs )
gdf_roads.to_file(f"{data_folder}/osm_data.shp")

In [3]:
gdf_roads= gpd.read_file(f'{data_folder}/osm_data.shp')

In [4]:
# Create preprocessing object and folder
gdf_roads = remove_unconnected_streets(gdf_roads.copy())
roundabout = gdf_roads[gdf_roads['junction'].isin(['roundabout', 'circular','mini_roundabout'])]
gdf_edges = gdf_roads[~gdf_roads['junction'].isin(['roundabout', 'circular','mini_roundabout'])]
# Load and filter the edges from the saved GeoPackage

gdf_edges = gdf_edges[~gdf_edges['highway'].str.contains('link', case=False, na=False)]
gdf_edges = gdf_edges.dropna(subset=['name'])
gdf_edges = gdf_edges[gdf_edges['name'] != '']
# Step 1: Merge connected lines with same 'name' and 'highway'
gdf_merge= merge_lines_by_attributes(gdf_edges, attrs=['name', 'highway'])
gdf_merge = gdf_merge[gdf_merge.geometry.notnull()]

# Step 2: Build internal intersection points efficiently
internal_points = extract_internal_intersections_spatial_index(list(gdf_merge.geometry))

# Step 3: Split merged lines by these points
first_results = split_lines_by_intersections_fast(gdf_merge, internal_points)
first_results['bearing'] = first_results['geometry'].apply(calculate_bearing)
first_results['angle'] = first_results['bearing'].apply(lambda x: x if x < 180 else x - 180)
first_results['length'] = first_results.length
first_results.to_file(f'{data_folder}/edges.shp')

Extracting intersections: 100%|██████████| 9209/9209 [00:01<00:00, 6197.39it/s]


In [5]:
def extract_deadend_points(gdf_edges):
    """
    Identifies all dead-end points (start or end points that appear only once in the entire edge dataset).

    Parameters:
        gdf_edges (GeoDataFrame): Edge lines with LineString geometries.

    Returns:
        Set of Points: All dead-end points.
    """
    endpoint_counter = defaultdict(int)

    for geom in gdf_edges.geometry:
        coords = list(geom.coords)
        endpoint_counter[coords[0]] += 1
        endpoint_counter[coords[-1]] += 1
    return endpoint_counter

def build_street_connection_dict(gdf_streets):
    """
    Build a dictionary of street-to-street connections via unique intersection points.

    Parameters:
        gdf_streets (GeoDataFrame): A GeoDataFrame containing street segments, with a 'name' column.

    Returns:
        connection_dict (dict): A dictionary where each key is a tuple of two street names (str, str),
                                and each value is a list of unique shapely Points where the streets intersect.
    """
    # Ensure clean index
    gdf = gdf_streets.reset_index(drop=True)

    # Step 1: Perform spatial join to find intersecting features
    joined = gpd.sjoin(gdf, gdf, how='inner', predicate='intersects')
    joined = joined[joined.index != joined['index_right']]  # Remove self-intersections

    # Step 2: Initialize output containers
    connection_dict = defaultdict(list)
    seen_edges = set()

    # Step 3: Process each intersecting pair
    for idx, row in tqdm(joined.iterrows(), total=len(joined)):
        idx2 = row['index_right']
        name1 = row['name_left']
        name2 = row['name_right']

        # Skip connections within the same street
        if name1 == name2:
            continue

        pair_key = tuple(sorted((name1, name2)))
        geom1 = row.geometry
        geom2 = gdf.loc[idx2].geometry
        intersection = geom1.intersection(geom2)

        # Skip if no actual intersection
        if intersection.is_empty:
            continue

        # Normalize intersection geometry into list of points
        if intersection.geom_type == 'Point':
            inter_points = [intersection]
        elif intersection.geom_type == 'MultiPoint':
            inter_points = list(intersection.geoms)
        else:
            continue  # Ignore non-point intersections

        # Step 4: Store only new intersection points
        existing_coords = {tuple(pt.coords)[0] for pt in connection_dict[pair_key]}
        new_points = [pt for pt in inter_points if tuple(pt.coords)[0] not in existing_coords]

        if new_points:
            connection_dict[pair_key].extend(new_points)
            seen_edges.add(pair_key)

    return connection_dict
conn_dict = build_street_connection_dict(first_results)

100%|██████████| 109104/109104 [00:08<00:00, 12575.11it/s]


In [6]:
# Merge roundabout geometries, identify which streets intersect them, and extract roundabout center points for further spatial analysis.


# --------------------`----------------------------------------
# STEP 1: Merge roundabout edges into clean geometries
# - Union geometries to dissolve boundaries
# - Linemerge to connect continuous segments
# - Export merged geometries as shapefile
# ------------------------------------------------------------
roundabout_edges = roundabout.copy()  # GeoDataFrame with roundabout segments

# Merge all geometries into a MultiLineString
merged_geom = unary_union(roundabout_edges.geometry)

# Connect continuous line segments
connected_lines = linemerge(merged_geom)

# Normalize to list of LineStrings
if isinstance(connected_lines, LineString):
    lines_list = [connected_lines]
elif isinstance(connected_lines, MultiLineString):
    lines_list = list(connected_lines.geoms)
else:
    lines_list = []

# Save merged roundabout geometries for inspection
gdf_roundabout = gpd.GeoDataFrame(geometry=lines_list, crs=roundabout_edges.crs)
gdf_roundabout.to_file(f'{data_folder}/roundabout.shp')

# ------------------------------------------------------------
# STEP 2: Create a buffer around merged roundabouts and find intersecting street edges
# ------------------------------------------------------------
# Buffer each merged roundabout line (e.g., 5 meters)
roundabout_buffer = gdf_roundabout.copy()
roundabout_buffer['geometry'] = roundabout_buffer.geometry.buffer(5)

# Perform spatial join: find street edges that intersect the roundabout buffer
intersections = gpd.sjoin(first_results, roundabout_buffer, how='inner', predicate='intersects')

# ------------------------------------------------------------
# STEP 3: Build dictionary mapping roundabout index to connected street names
# ------------------------------------------------------------
roundabout_to_streets = defaultdict(set)

for _, row in intersections.iterrows():
    roundabout_idx = row['index_right']  # index of the matched roundabout in gdf_merged
    street_name = row['name']
    roundabout_to_streets[roundabout_idx].add(street_name)

# Save the original roundabout edge segments for reference
roundabout_edges.to_file(f'{data_folder}/roundabout_edges.shp')

# ------------------------------------------------------------
# STEP 4: Calculate center point for each roundabout polygon buffer
# - Buffer again with clean style, then extract centroid
# ------------------------------------------------------------
# Buffer with styling for rounded shapes, then get centroids
roundabout_centers = gdf_roundabout['geometry'].buffer(distance=1, cap_style=1, join_style=1).centroid

# Compute distance from centroid to the boundary of the buffer
distances = [
    centroid.distance(line)
    for centroid, line in zip(roundabout_centers, gdf_roundabout['geometry'])
]

# Save as GeoDataFrame
gdf_roundabouts = gpd.GeoDataFrame(geometry=roundabout_centers, crs=roundabout_edges.crs)
# Compute distance from centroid to buffer boundary
gdf_roundabouts['dist'] = distances
gdf_roundabouts.reset_index(drop=True).to_file(f'{data_folder}/center_roundabout.shp')



In [111]:

# This code performs the main part of the simplification
# --- Helper Functions ---

def check_parallelism(to_translate: GeoDataFrame) -> bool:
    """
    Checks whether a group of line segments contain any parallel segments
    by offsetting each line and checking buffer intersections.
    """
    my_buffer = to_translate['geometry'].buffer(cap_style=2, distance=30, join_style=3)
    to_translate['geometry_right'] = to_translate['geometry'].apply(lambda x: x.parallel_offset(35, 'right'))
    to_translate['geometry_left'] = to_translate['geometry'].apply(lambda x: x.parallel_offset(35, 'left'))

    def is_parallel(my_s_join: GeoDataFrame, the_buffer, geo_field: str):
        my_s_join['geometry'] = my_s_join[geo_field]
        sjoin = my_s_join.sjoin(GeoDataFrame(geometry=the_buffer, crs=project_crs), how='inner')
        sjoin = sjoin[sjoin.index != sjoin['index_right']]
        for _, row in sjoin.iterrows():
            overlay = gpd.overlay(
                GeoDataFrame([row], crs=project_crs),
                GeoDataFrame(geometry=[the_buffer[row['index_right']]], crs=project_crs),
                how='intersection')
            if (overlay.length / row.geometry.length).iloc[0] * 100 > 10:
                return True
        return False

    return is_parallel(to_translate, my_buffer, 'geometry_right') or is_parallel(to_translate, my_buffer, 'geometry_left')

def circular_distance(angle1, angle2):
    """Compute minimum circular angle difference between two angles."""
    diff = np.abs(angle1 - angle2) % 180
    return np.minimum(diff, 180 - diff)

def add_more_pnts_to_new_lines(pnt_f, pnt_l, line_pnts, lngth_chck, test_poly):
    """
    Recursively add points along a line if they're more than 10m from existing road segments.
    """
    dist = pnt_f.distance(pnt_l)
    x0, y0 = pnt_f.x, pnt_f.y
    bearing = math.atan2(pnt_l.x - x0, pnt_l.y - y0)
    if bearing < 0:
        bearing += 2 * math.pi
    loops = int(dist / lngth_chck)

    for step in range(1, loops):
        x_new = x0 + lngth_chck * step * math.sin(bearing)
        y_new = y0 + lngth_chck * step * math.cos(bearing)
        new_point = Point(x_new, y_new)
        nearest = GeoDataFrame(geometry=[new_point], crs=project_crs).sjoin_nearest(test_poly, distance_col='dis').iloc[0]
        if nearest['dis'] > 10:
            line = data.loc[nearest['index_right']]['geometry']
            projected = line.interpolate(line.project(new_point))
            if projected.distance(pnt_f) < 10:
                continue
            line_pnts.append(projected)
            return add_more_pnts_to_new_lines(projected, pnt_l, line_pnts, lngth_chck, test_poly)
    return line_pnts

def create_center_line(one_poly):
    """
    Construct a center line through a polygon by identifying its farthest endpoints
    and interpolating new points as needed based on angular continuity.
    """
    lines = data.sjoin(GeoDataFrame(geometry=[one_poly], crs=project_crs)).drop(columns='index_right')
    endpoints = []
    lines['geometry'].apply(lambda line: endpoints.extend([Point(line.coords[0]), Point(line.coords[-1])]))
    combos = list(combinations(endpoints, 2))

    df = DataFrame({
        'point_1': [a for a, _ in combos],
        'point_2': [b for _, b in combos],
    })
    df['dist'] = df.apply(lambda x: x['point_1'].distance(x['point_2']), axis=1)
    dx = df['point_2'].apply(lambda p: p.x) - df['point_1'].apply(lambda p: p.x)
    dy = df['point_2'].apply(lambda p: p.y) - df['point_1'].apply(lambda p: p.y)
    df['angle'] = np.degrees(np.arctan2(dy, dx)) % 180
    avg_angle = lines['angle'].mean()
    df['ratio'] = df['dist'] / df['dist'].max() + 0.5 * np.abs(df['angle'] - avg_angle) / np.abs(df['angle'] - avg_angle).max()

    pnt_f, pnt_l = df.sort_values(by='ratio', ascending=False).iloc[0][['point_1', 'point_2']]
    angle_range = lines['angle'].max() - lines['angle'].min()

    if angle_range < 1:
        new_line_pts = [pnt_f]
    else:
        step = 8.5 if angle_range > 100 else 75 - log2(angle_range) * 10
        new_line_pts = add_more_pnts_to_new_lines(pnt_f, pnt_l, [pnt_f], step, lines)
    new_line_pts.append(pnt_l)
    return new_line_pts

def update_df_with_center_line(new_line, is_simplified=0, group_name=-1):
    """Append a new line feature to the output dictionary."""
    dic_final['name'].append(name)
    dic_final['geometry'].append(new_line)
    dic_final['highway'].append(data.iloc[0]['highway'])
    dic_final['bearing'].append(data['angle'].mean())
    dic_final['group'].append(group_name)
    dic_final['is_simplified'].append(is_simplified)

# --- Main logic ---
dic_final = {'name': [], 'geometry': [], 'highway': [], 'bearing': [], 'group': [], 'is_simplified': []}
df_pro = first_results.copy()
grouped = df_pro.groupby('name')

with tqdm(total=len(grouped)) as pbar:
    for name, group_df in grouped:
        pbar.update(1)
        group_df = group_df.dropna(subset=['angle'])
        if len(group_df) < 2:
            data = group_df
            _ = group_df['geometry'].apply(lambda geom: update_df_with_center_line(geom))
            continue

        angles = group_df['angle'].to_numpy()
        dists = np.array([[circular_distance(a1, a2) for a2 in angles] for a1 in angles])
        dbscan = DBSCAN(eps=10, min_samples=2, metric='precomputed')
        group_df['group'] = dbscan.fit_predict(dists)

        if (group_df['group'] == -1).all():
            data = group_df
            _ = group_df['geometry'].apply(lambda geom: update_df_with_center_line(geom))
            continue

        for group_id, sub_group in group_df.groupby('group'):
            data = sub_group
            if group_id == -1:
                _ = data['geometry'].apply(lambda geom: update_df_with_center_line(geom))
                continue
            if check_parallelism(data.copy()):
                min_polylines = len(data) / 15
                condition = (data['highway'].isin(['service', 'unclassified'])) & (
                    data.groupby('highway')['highway'].transform('count') <= min_polylines)
                data = data[~condition]

                buffers = data.buffer(cap_style=3, distance=30, join_style=3)
                unified = buffers.unary_union

                if isinstance(unified, MultiPolygon):
                    for poly in unified.geoms:
                        center_pts = create_center_line(poly)
                        update_df_with_center_line(LineString(center_pts), 1, group_id)
                else:
                    center_pts = create_center_line(unified)
                    update_df_with_center_line(LineString(center_pts), 1, group_id)
            else:
                _ = data['geometry'].apply(lambda geom: update_df_with_center_line(geom))

# Finalize and export
print(f'number_of_parallel: {sum(dic_final["is_simplified"])}')
print('create new files')
new_network = GeoDataFrame(dic_final, crs=project_crs)
new_network['length'] = new_network.length
new_network.to_file(f'{data_folder}/simp.shp')


100%|██████████| 3177/3177 [02:49<00:00, 18.76it/s]


number_of_parallel: 1252
create new files


In [7]:
new_network = gpd.read_file(f'{data_folder}/simp.shp')

In [8]:

# -----------------------------
# Simplify and Clean Network
# -----------------------------

def simplify_and_clean_street_network(new_network, roundabout_to_streets):
    """
    Cleans and simplifies a street network by:
    - Removing short terminal lines that are weakly connected.
    - Ensuring intersections occur at endpoints.
    - Eliminating redundant coordinates.

    Parameters:
        new_network (GeoDataFrame): The initial simplified network.
        roundabout_to_streets (dict): Mapping of roundabout IDs to connected street names.

    Returns:
        GeoDataFrame: Cleaned and simplified network.
    """

    gdf = new_network.copy()
    # gdf =gdf[gdf.length>10]

    # --------------------------------------------
    # Step 1: Precompute useful attributes
    # --------------------------------------------

    gdf['name_count'] = gdf['name'].map(gdf['name'].value_counts())

    # Count endpoint appearances across all geometries
    endpoint_counts = defaultdict(int)
    for geom in gdf.geometry:
        coords = list(geom.coords)
        endpoint_counts[coords[0]] += 1
        endpoint_counts[coords[-1]] += 1

    # --------------------------------------------
    # Step 2: Filter for lines connected to roundabouts
    # --------------------------------------------

    connected_streets = {name for names in roundabout_to_streets.values() for name in names}
    gdf_connected = gdf[gdf['name'].isin(connected_streets)].copy()

    # Flag short lines with only one connected endpoint
    lines_to_remove = set(gdf_connected[
        (gdf_connected['length'] < 30) &
        (gdf_connected['name_count'] > 1) &
        (gdf_connected.geometry.apply(lambda geom: endpoint_counts[geom.coords[0]] == 1 and
                                                      endpoint_counts[geom.coords[-1]] == 1))
    ].index)

    # --------------------------------------------
    # Step 3: Identify short, isolated terminal lines
    # --------------------------------------------

    candidates = gdf_connected[
        (gdf_connected['length'] < 100) &
        (gdf_connected['name_count'] > 1) &
        (gdf_connected.geometry.apply(lambda geom: endpoint_counts[geom.coords[0]] == 1 and
                                                      endpoint_counts[geom.coords[-1]] == 1))
    ].copy()

    # --------------------------------------------
    # Step 4: Find actual intersections (excluding self-intersection)
    # --------------------------------------------

    intersections = gpd.sjoin(candidates, gdf, how='inner', predicate='intersects')
    intersections = intersections[intersections.index != intersections['index_right']]

    # --------------------------------------------
    # Step 5: Determine which candidates are not validly connected
    # --------------------------------------------

    for idx, row in candidates.iterrows():
        temp_line = row.geometry
        start, end = Point(temp_line.coords[0]), Point(temp_line.coords[-1])
        matching = intersections.loc[intersections.index == idx]

        if matching.empty:
            lines_to_remove.add(idx)
            continue

        # Check if intersection occurs exactly at an endpoint
        valid = False
        for other_idx in matching['index_right']:
            inter = temp_line.intersection(gdf.loc[other_idx].geometry)
            if inter.is_empty:
                continue

            # Normalize to list of points
            inter_points = [inter] if isinstance(inter, Point) else list(inter.geoms)
            if any(pt.equals(start) or pt.equals(end) for pt in inter_points):
                valid = True
                break

        if not valid:
            lines_to_remove.add(idx)

    # --------------------------------------------
    # Step 6: Remove flagged lines and clean geometry
    # --------------------------------------------

    gdf_simplified = gdf.drop(index=lines_to_remove).copy()

    # Remove duplicate consecutive points in LineString
    gdf_simplified['geometry'] = gdf_simplified['geometry'].apply(
        lambda geom: LineString([pt for pt, _ in groupby(geom.coords)]) if len(set(geom.coords)) > 1 else None
    )

    # Drop lines with invalid or null geometries
    gdf_simplified = gdf_simplified[gdf_simplified['geometry'].notnull()].copy()

    return gdf_simplified
# gdf_edges = merge_lines_by_attributes(new_network.copy(), attrs=['name', 'highway'])
#
# gdf_edges = gdf_edges[gdf_edges.geometry.notnull()]
#
# # Step 2: Build internal intersection points efficiently
# internal_points = extract_internal_intersections_spatial_index(list(gdf_edges.geometry))
#
# # Step 3: Split merged lines by these points
# gdf_edges = split_lines_by_intersections_fast(gdf_edges, internal_points)
gdf_edges= new_network.copy()
gdf_edges['length'] = gdf_edges.length
gdf_simplified = simplify_and_clean_street_network(gdf_edges, roundabout_to_streets)
gdf_simplified.to_file(f'{test_folder}/gdf_simplified_fix0.shp')

In [114]:
### Test Area

In [115]:
### Test Area -  END

In [87]:
 # This code is enhancing the geometry of a street network by snapping street segments to nearby roundabouts to ensure cleaner, more topologically correct connections.

# --- Parameters ---
  # Threshold to treat the roundabout as coincident with a vertex

# --- Working copy ---
gdf_simplified_fix1 = gdf_simplified.copy()
existing_connections = defaultdict(list)



# --- Iterate roundabouts and their connected streets ---
for ridx, streets in roundabout_to_streets.items():
    roundabout_point = gdf_roundabouts.geometry.iloc[ridx]
    roundabout_dist = gdf_roundabouts['dist'].iloc[ridx]
    TOLERANCE_NEAR_VERTEX = roundabout_dist *1.5
    for street in streets:
        street_edges = gdf_simplified_fix1[gdf_simplified_fix1['name'] == street]
        if street_edges.empty:
            continue

        # Compute distances from each edge to the roundabout
        dists = street_edges.geometry.distance(roundabout_point)
        min_dist = dists.min()
        closest_edges = street_edges[dists == min_dist]

        for edge_idx, edge_row in closest_edges.iterrows():
            line = edge_row.geometry
            new_coords = list(line.coords)

            # Project the roundabout point onto the line
            nearest_pt = line.interpolate(line.project(roundabout_point))
            p_start = Point(new_coords[0])
            p_end = Point(new_coords[-1])

            round_coord = roundabout_point.coords[0]

            # --- Insertion logic ---
            # Find the closest existing vertex on the line
            insert_idx = np.argmin([Point(c).distance(nearest_pt) for c in new_coords])
            rpt = Point(new_coords[insert_idx])
            def dist(p1, p2):
                return np.linalg.norm(np.array(p1) - np.array(p2))
            dist_to_start = dist(new_coords[insert_idx], new_coords[0])
            dist_to_end = dist(new_coords[insert_idx], new_coords[-1])
            # Efficient conditional logic
            if dist_to_start < 30 and dist_to_end < 30:
                if dist_to_start < dist_to_end:
                    rpt, insert_idx = p_start, 0
                else:
                    rpt, insert_idx = p_end, len(new_coords) - 1
            elif dist_to_start < 30:
                rpt, insert_idx = p_start, 0
            elif dist_to_end < 30:
                rpt, insert_idx = p_end, len(new_coords) - 1
            if insert_idx == 0 and roundabout_point.distance(rpt) < TOLERANCE_NEAR_VERTEX and all(not geom.contains(rpt) for geom in gdf_roundabouts.geometry.buffer(1)):
                    new_coords = [round_coord] + new_coords[1:]
            elif insert_idx == len(new_coords) - 1 and roundabout_point.distance(rpt) < TOLERANCE_NEAR_VERTEX and all(not geom.contains(rpt) for geom in gdf_roundabouts.geometry.buffer(1)):
                    new_coords = new_coords[:-1] + [round_coord]
            elif roundabout_point.distance(rpt) < TOLERANCE_NEAR_VERTEX and all(not geom.contains(rpt) for geom in gdf_roundabouts.geometry.buffer(1)):
                new_coords = new_coords[:insert_idx] + [round_coord] + new_coords[insert_idx + 1:]
            else:
                # Check angle between segments to ensure smooth connection
                def bearing(p1, p2):
                    dx, dy = p2[0] - p1[0], p2[1] - p1[1]
                    return np.degrees(np.arctan2(dy, dx)) % 360
                if insert_idx==0:
                    az1 = bearing(new_coords[insert_idx], new_coords[insert_idx +1])
                    az2 = bearing(new_coords[insert_idx], round_coord)
                else:
                    az1 = bearing(new_coords[insert_idx - 1], new_coords[insert_idx])
                    az2 = bearing(new_coords[insert_idx], round_coord)
                angle_diff = abs((az1 - az2 + 180) % 360 - 180)
                if 110 <= angle_diff <= 250:
                    new_coords = new_coords[:insert_idx] + [round_coord] + new_coords[insert_idx:]
                else:
                    new_coords = new_coords[:insert_idx + 1] + [round_coord] + new_coords[insert_idx + 1:]


            gdf_simplified_fix1.at[edge_idx,'geometry'] = LineString(new_coords)
            existing_connections[(ridx,street)].append(edge_idx)

# --- Apply updated geometries to GeoDataFrame ---
gdf_simplified_fix1.drop(columns='coords', inplace=True, errors='ignore')
gdf_simplified_fix1['length'] = gdf_simplified_fix1.length
gdf_simplified_fix1.reset_index().to_file(f'{test_folder}/gdf_simplified_fix1.shp')


In [50]:
### TEST AREA ####

In [51]:

### TEST AREA  END###

In [88]:
def compute_azimuth(p1, p2):
    dx, dy = p2[0] - p1[0], p2[1] - p1[1]
    return np.degrees(np.arctan2(dy, dx)) % 360

def angle_difference(az1, az2):
    diff = abs(az1 - az2)
    return min(diff, 360 - diff)
def trim_roundabout_point(coords, rcoord,test_all=True,old_pnt=None):
    """
    Removes the roundabout coordinate from the coordinate list if it exists.
    This function supports cases where the roundabout is not necessarily at the start or end.
    """
    if len(coords) < 3:
        return coords
    if test_all:
        return [pt for pt in coords if pt != rcoord]
    if coords[0] == rcoord:
        coords[0] = old_pnt
        return coords
    elif coords[-1] == rcoord:
        coords[-1] = old_pnt
        return coords
    return -1



def snap_dead_ends_with_angle_filter(
    gdf_streets,
    gdf_roundabouts,
    roundabout_to_streets,
    existing_connections,
    tolerance=120
):
    gdf_fixed = gdf_streets.copy()
    node_count = defaultdict(int)

    for _, row in tqdm(gdf_fixed.iterrows(), total=len(gdf_fixed)):
        coords = list(row.geometry.coords)
        node_count[coords[0]] += 1
        node_count[coords[-1]] += 1
    for pt in gdf_roundabouts.geometry:
        node_count[pt.coords[0]] += 1

    lines_to_drop = set()

    for ridx, streets in roundabout_to_streets.items():
        roundabout_row = gdf_roundabouts.loc[ridx]
        rpt = roundabout_row.geometry
        rcoord = rpt.coords[0]
        rbuffer = rpt.buffer(roundabout_row['dist'] * 2)
        for street in streets:
            street_edges = gdf_fixed[gdf_fixed["name"] == street]

            for idx, row in street_edges.iterrows():
                if idx in existing_connections[(ridx, street)]:
                    continue
                coords = list(row.geometry.coords)
                start, end = Point(coords[0]), Point(coords[-1])
                snapped = False

                if rbuffer.contains(start) and node_count[coords[0]] == 1:
                    old_pnt = coords[0]
                    coords[0] = rcoord
                    snapped = True
                elif rbuffer.contains(end) and node_count[coords[-1]] == 1:
                    old_pnt = coords[-1]
                    coords[-1] = rcoord
                    snapped = True

                if not snapped:
                    continue

                new_line = LineString(coords)
                new_is_start = coords[0] == rcoord
                new_az = compute_azimuth(rcoord, coords[1] if new_is_start else coords[-2])
                redundant = False

                for ex_id in existing_connections.get((ridx, street), []):
                    if ex_id == idx:
                        break
                    ex_geom = gdf_fixed.loc[ex_id].geometry
                    ex_coords = list(ex_geom.coords)

                    # Remove roundabout from new and existing line
                    trimmed_new = LineString(trim_roundabout_point(coords, rcoord,False,old_pnt))

                    trimmed_existing =  LineString(trim_roundabout_point(ex_coords, rcoord))

                    # Distance check
                    if trimmed_new.distance(trimmed_existing) > 50:
                        existing_connections[(ridx, street)].append(idx)
                        gdf_fixed.at[idx, 'geometry'] = new_line
                        redundant = True
                    else:
                        ex_is_start = ex_coords[0] == rcoord
                        ex_az = compute_azimuth(rcoord, ex_coords[1] if ex_is_start else ex_coords[-2])
                        az_diff = angle_difference(new_az, ex_az)

                        if az_diff < tolerance:
                            if new_line.length > ex_geom.length:
                                lines_to_drop.add(ex_id)
                                existing_connections[(ridx, street)].remove(ex_id)
                                existing_connections[(ridx, street)].append(idx)
                                gdf_fixed.at[idx, 'geometry'] = new_line
                            else:
                                lines_to_drop.add(idx)
                            redundant = True
                            break

                if not redundant:
                    existing_connections[(ridx, street)].append(idx)
                    gdf_fixed.at[idx, 'geometry'] = new_line


    return gdf_fixed.drop(index=list(lines_to_drop)).copy()
gdf_simplified_fix2= snap_dead_ends_with_angle_filter(gdf_simplified_fix1,gdf_roundabouts,roundabout_to_streets,
    existing_connections)
gdf_simplified_fix2.to_file(f'{test_folder}/gdf_simplified_fix2.shp')

100%|██████████| 14532/14532 [00:01<00:00, 13308.96it/s]


In [120]:
#### TEST AREA

In [121]:
### TEAT AREA - END

In [89]:
# This code removes duplicate street segments (with same endpoints but different geometries) around roundabouts, keeping only the longest one per pair.


# Make a working copy of the dataset
gdf_simplified_fix3 = gdf_simplified_fix2.copy()

# Set to store indices of duplicate edges to remove
duplicate_removal_indices = set()

# --- Helper function: Normalize edge as unordered pair of start/end points ---
def normalize_edge(pt1, pt2):
    return tuple(sorted([tuple(pt1), tuple(pt2)]))

# --- Step 1: Loop through roundabouts and their connected streets ---
for ridx, streets in roundabout_to_streets.items():
    # Collect all edges from the connected streets
    street_edges = gdf_simplified_fix3[gdf_simplified_fix3['name'].isin(streets)]

    # Group edges by normalized endpoint pair (to catch reversed duplicates)
    edge_groups = defaultdict(list)

    for idx, row in street_edges.iterrows():
        coords = list(row.geometry.coords)
        edge_key = normalize_edge(coords[0], coords[-1])
        edge_groups[edge_key].append((idx, row.geometry.length))

    # --- Step 2: Within each group, retain only the longest edge ---
    for edges in edge_groups.values():
        if len(edges) > 1:
            # Sort by length in descending order
            edges_sorted = sorted(edges, key=lambda x: -x[1])
            # Keep the longest (first); mark others for removal
            for edge_idx, _ in edges_sorted[1:]:
                duplicate_removal_indices.add(edge_idx)

# --- Step 3: Remove marked duplicates and export result ---
gdf_simplified_fix3 = gdf_simplified_fix3[~gdf_simplified_fix3.index.isin(duplicate_removal_indices)].copy()
gdf_simplified_fix3.to_file(f'{data_folder}/gdf_fix_ra.shp')



In [90]:
## Test for the roundabout
joined = gpd.sjoin(gdf_roundabouts, gdf_simplified_fix3, how='left', predicate='intersects')

def compare_roundabout_dicts(old_dict, new_dict):
    differences = {}

    all_keys = set(old_dict.keys()).union(new_dict.keys())

    for key in all_keys:
        old = set(old_dict.get(key, []))
        new = set(new_dict.get(key, []))

        if old != new:
            differences[key] = {
                'added': new - old,
                'removed': old - new
            }

    return differences
roundabout_to_streets_new = defaultdict(set)

for ridx, row in joined.iterrows():
    roundabout_idx = row.name  # index of the roundabout
    street_name = row['name']
    if pd.notnull(street_name):
        roundabout_to_streets_new[roundabout_idx].add(street_name)
diffs = compare_roundabout_dicts(roundabout_to_streets, roundabout_to_streets_new)

if not diffs:
    print("✅ Roundabout-street mapping is unchanged.")
else:
    print("❌ Differences found:")
    for ridx, change in diffs.items():
        print(f"\nRoundabout {ridx}:")
        if change['added']:
            print(f"  ➕ Streets added: {sorted(change['added'])}")
        if change['removed']:
            print(f"  ➖ Streets removed: {sorted(change['removed'])}")



❌ Differences found:

Roundabout 117:
  ➖ Streets removed: ['Via Druento']


In [91]:
conn_dict2 = build_street_connection_dict(gdf_simplified_fix3)

def compare_connection_dicts(original_connection_dict, updated_connection_dict):
    """
    Compare two street connection dictionaries. Identify connections that existed in the original
    version but are missing in the updated version.

    Parameters:
        original_connection_dict (dict): { (street1, street2): [Point, ...] }
        updated_connection_dict (dict): { (street1, street2): [Point, ...] }

    Returns:
        missing_connections (dict): connections from the original missing in the updated
    """
    missing_connections = {}

    for conn_pair, orig_points in original_connection_dict.items():
        if conn_pair not in updated_connection_dict:
            missing_connections[conn_pair] = orig_points
        else:
            # Optionally, you can check if some intersection points are missing
            updated_coords = {tuple(pt.coords)[0] for pt in updated_connection_dict[conn_pair]}
            missing_pts = [pt for pt in orig_points if tuple(pt.coords)[0] not in updated_coords]
            if missing_pts:
                missing_connections[conn_pair] = missing_pts

    return missing_connections

# Example usage
missing = compare_connection_dicts(conn_dict, conn_dict2 )

# Print or log missing entries
for pair, pts in missing.items():
    print(f"Missing connection between {pair[0]} and {pair[1]} at {len(pts)} point(s)")


100%|██████████| 52286/52286 [00:07<00:00, 6623.09it/s]


Missing connection between Sottopassaggio del Lingotto and Via Nizza at 6 point(s)
Missing connection between Corso Caduti sul lavoro and Via Nizza at 3 point(s)
Missing connection between Ciclabile Regione Piemonte - Via Sommariva and Via Nizza at 2 point(s)
Missing connection between Via Nizza and Via Valenza at 2 point(s)
Missing connection between Via Farigliano and Via Nizza at 1 point(s)
Missing connection between Via Nizza and Via Sommariva at 1 point(s)
Missing connection between Via Alba and Via Nizza at 1 point(s)
Missing connection between Via Nizza and Via Vinovo at 1 point(s)
Missing connection between Via Nizza and Via Passo Buole at 1 point(s)
Missing connection between Via Felizzano and Via Nizza at 1 point(s)
Missing connection between Via Nizza and Via Testona at 1 point(s)
Missing connection between Via Beinette and Via Nizza at 1 point(s)
Missing connection between Corso Piero Maroncelli and Via Nizza at 3 point(s)
Missing connection between Piazza Giosuè Carducci a

In [92]:
def filter_intersection_points(points, tolerance=70):
    """
    Filters a list of intersection points based on spatial proximity.

    Logic:
    - If only 1 or 0 points: return as-is with empty representative mapping.
    - If 2 points: if close (< tolerance), return one and map both to it.
    - If >2: use DBSCAN to group points, select one per group, and store mapping from all grouped points to representative.

    Ensures the representative point is not duplicated in its own value list.

    Parameters:
        points (list of shapely.geometry.Point)
        tolerance (float): distance threshold in meters for clustering

    Returns:
        tuple:
            - list of representative points (filtered)
            - dict: {representative Point: [all merged Point(s) excluding rep]}
    """
    if len(points) <= 1:
        return points, {}

    if len(points) == 2:
        if points[0].distance(points[1]) < tolerance:
            return [points[0]], {points[0]: [points[1]]}
        else:
            return points, {}

    # More than two points → use DBSCAN
    coords = np.array([[pt.x, pt.y] for pt in points])
    db = DBSCAN(eps=tolerance-10, min_samples=1).fit(coords)
    labels = db.labels_

    result = []
    group_map = defaultdict(list)

    label_to_rep = {}
    for label in np.unique(labels):
        rep_idx = np.where(labels == label)[0][0]
        rep = points[rep_idx]
        label_to_rep[label] = rep
        result.append(rep)

    for pt, label in zip(points, labels):
        rep = label_to_rep[label]
        if pt != rep:
            group_map[rep].append(pt)

    return result, group_map


In [93]:

def find_nearest_line_from_list(lines: list, point: Point):
    tree = STRtree(lines)
    nearest_geom = tree.nearest(point)
    return nearest_geom

def get_closest_endpoint_to_line(line_a: LineString, line_b: LineString):
    """Returns the endpoint of line_a closest to line_b and its distance."""
    endpoints = [Point(line_a.coords[0]), Point(line_a.coords[-1])]
    distances = [pt.distance(line_b) for pt in endpoints]
    return np.argmin(distances),distances[np.argmin(distances)]

def extend_line_to_target(line: LineString, endpoint_index: int, target: LineString) -> LineString:
    """
    Extend a line by projecting its start or end point onto a target line.

    Parameters:
        line (LineString): The line to extend.
        endpoint_index (int): Use 0 to extend from start, -1 from end.
        target (LineString): The line to project the endpoint onto.

    Returns:
        LineString: Extended line.
    """
    coords = list(line.coords)

    if len(coords) < 2:
        return line  # Can't extend a single-point line

    # Get reference segment
    if endpoint_index == 0:
        p2 = Point(coords[0])
    else:
        p2 = Point(coords[-1])

    # Get projected point from p2 to target
    proj_distance = target.project(p2)
    projected_point = target.interpolate(proj_distance)

    # Extend line to this projected point
    if endpoint_index == 0:
        new_coords = [projected_point.coords[0]] + coords
    else:
        new_coords = coords + [projected_point.coords[0]]

    return LineString(new_coords)

def connect_lines(line1, line2):
    """
    Returns a modified line1 or line2 extended to intersect with the other.
    """
    lg1  = line1.geometry
    lg2 = line2.geometry
    idx1, dist1 = get_closest_endpoint_to_line(lg1, lg2)
    idx2, dist2 = get_closest_endpoint_to_line(lg2, lg1)

    if dist1 < dist2:
        return extend_line_to_target(lg1, idx1,lg2), line1.name

    else:
        return extend_line_to_target(lg2, idx2,lg1), line2.name

def connect_nearest_lines(lines1, lines2, point, gdf_streets):
    """
    Finds the nearest line in each set to a given point, connects them using a custom function,
    and updates the geometry in gdf_streets.

    Parameters:
        lines1 (GeoDataFrame): First set of candidate lines (subset of gdf_streets).
        lines2 (GeoDataFrame): Second set of candidate lines (subset of gdf_streets).
        point (shapely.geometry.Point): The reference point to determine nearest lines.
        gdf_streets (GeoDataFrame): The full street dataset to be updated.
        connect_lines_func (function): A function that takes two lines and returns (new_line, index_to_update).

    Returns:
        GeoDataFrame: Updated gdf_streets with the new connected line geometry.
    """

    def get_nearest_line(lines, pt):
        return lines.iloc[find_nearest_line_from_list(list(lines.geometry), pt)] if len(lines) > 1 else lines.iloc[0]

    line1 = get_nearest_line(lines1, point)
    line2 = get_nearest_line(lines2, point)
    new_line,new_idx = connect_lines(line1,line2)
    gdf_streets.at[new_idx, 'geometry'] = new_line
    return gdf_streets



In [94]:

def find_nearest_intersections_overlay(gdf1, gdf2, ref_pnt):
    """
    Vectorized approach: find the minimum distance from each gdf1 line to the nearest intersection
    with lines in gdf2, measured from a reference point on each gdf1 line.

    Parameters:
        gdf1 (GeoDataFrame): GeoDataFrame with LineString geometries.
        gdf2 (GeoDataFrame): Another GeoDataFrame with LineString geometries.
        ref (str): Reference point on gdf1 geometry. Options: 'centroid', 'start', or 'end'.

    Returns:
        list of float: Distances from each gdf1 feature to its nearest intersection point, or 35 if no intersection.
        :param ref_pnt:
    """

    gdf1['geometry'] = gdf1['geometry'].buffer(0.1)
    gdf2['geometry'] =  gdf1['geometry'].buffer(0.1)
    intersections = gpd.overlay(gdf1, gdf2, how='intersection')


    if intersections.empty:
        return 35
    # Map reference points to intersections
        # Compute distance to a given point (pt_coords should be a shapely.geometry.Point)
    intersections['distance'] = intersections.geometry.distance(ref_pnt)
    return intersections['distance'].min()


In [95]:

TOLERANCE_NEAREST = 30

# Main loop
def snap_missing_points_to_streets(gdf_streets, missing):
    for (street1, street2), points_0 in tqdm(missing.items(), total=len(missing)):

        points, points_map = filter_intersection_points(points_0)
        lines1 = gdf_streets[gdf_streets['name'] == street1].copy()
        lines2 = gdf_streets[gdf_streets['name'] == street2].copy()
        if len(points)==1:
            if len(lines1.sjoin(lines2))>0:
                continue
            else:
                gdf_streets= connect_nearest_lines(lines1, lines2, points[0], gdf_streets)
        else:
            for pt in points:
                if find_nearest_intersections_overlay(lines1.copy(),lines2.copy(),pt) < TOLERANCE_NEAREST:
                    continue
                else:
                    gdf_streets= connect_nearest_lines(lines1.copy(), lines2.copy(), pt, gdf_streets)
    return gdf_streets

gdf_fixed = snap_missing_points_to_streets(gdf_simplified_fix3.copy(), missing)
gdf_fixed.reset_index().to_file(f'{data_folder}/network_final.shp')


100%|██████████| 3423/3423 [00:42<00:00, 80.20it/s] 


In [185]:
gdf_fixed = gpd.read_file(f'{data_folder}/network_final.shp').set_index('index')


In [186]:

def _extend_to_target(pt_deadend: Point,
                      pt_prev: Point,
                      gdf_all: gpd.GeoDataFrame,
                      sindex,
                      cidx,
                      max_length: float = 100):
    """
    Shoot a ray from the dead-end point in the direction of (pt_prev → pt_deadend)
    up to `max_length` and return a new point where it first meets another edge.
    If nothing is hit, return the projection of the dead-end onto the nearest edge.
    """
    # Direction vector (unit)
    vec = np.array(pt_deadend.coords[0]) - np.array(pt_prev.coords[0])
    norm = np.linalg.norm(vec)
    if norm == 0:         # safety
        return pt_deadend
    vec_unit = vec / norm

    ray_end = Point(pt_deadend.x + vec_unit[0] * max_length,
                    pt_deadend.y + vec_unit[1] * max_length)
    ray = LineString([pt_deadend, ray_end])

    # Spatial index for speed
    hits = list(sindex.intersection(ray.bounds))
    candidates0 = gdf_all.iloc[hits]
    candidates = candidates0[candidates0.index != cidx].geometry
    # First intersection (if any)
    intersection_pts = [ray.intersection(g) for g in candidates if ray.intersects(g)]
    intersection_pts = [pt for pt in intersection_pts if not pt.is_empty]

    if intersection_pts:
        # Pick the closest intersection to the dead-end
        dists = [pt_deadend.distance(pt) for pt in intersection_pts]
        return intersection_pts[int(np.argmin(dists))]
    else:
        # Fall-back: project to closest edge (excluding the current ray)
        try:
            other_edges = gdf_all[~gdf_all.geometry.equals(pt_deadend)]
        except KeyError:
            print(cidx)
        nearest_geom = other_edges.geometry.unary_union
        proj = nearest_points(pt_deadend, nearest_geom)[1]
        return proj

def clean_and_fix_deadends(
        gdf_edges: gpd.GeoDataFrame,
        preserved_old: set,
        preserved_new: set,
        *,
        buffer_close: float = 20,
        short_cut: float = 50,
        extend_max: float = 100
) -> gpd.GeoDataFrame:
    """
    • Adds columns: start_deadend, end_deadend, has_deadend
    • Drops edges with dead-ends at *both* ends
    • For one-sided dead-ends:
        – If the dead-end is within `buffer_close` of any preserved_old → keep
        – Else, if length < `short_cut` → drop
        – Else → extend until it meets another edge (or nearest edge if none
          within `extend_max`)
    """
    # --- 1. Annotate dead-end status ----------------------------------------
    def _is_deadend(pt):
        return pt in preserved_new

    gdf = gdf_edges.copy()
    sindex = gdf.sindex
    gdf["start_deadend"] = gdf.geometry.apply(lambda g: _is_deadend(g.coords[0]))
    gdf["end_deadend"]   = gdf.geometry.apply(lambda g: _is_deadend(g.coords[-1]))
    gdf["has_deadend"]   = gdf["start_deadend"] | gdf["end_deadend"]

    # Split: rows with / without dead-ends
    keep_rows = gdf[~gdf.has_deadend].copy()
    fix_rows  = gdf[gdf.has_deadend].copy()

    fixed_geoms = []

    for idx, row in tqdm(fix_rows.iterrows(), total=len(fix_rows), desc="Processing edges"):
        geom = row.geometry
        coords = list(geom.coords)
        start_pt, end_pt = Point(coords[0]), Point(coords[-1])
        start_de, end_de = row.start_deadend, row.end_deadend

        # --- 2. Remove if dead-end at both ends -----------------------------
        if start_de and end_de:
            continue  # DROP edge entirely

        # Identify the dead-end & its previous vertex
        if start_de:
            de_pt, prev_pt = start_pt, Point(coords[1])
        else:
            de_pt, prev_pt = end_pt, Point(coords[-2])

        # --- 3. Close to preserved_old? -------------------------------------
        if any(de_pt.distance(Point(old)) < buffer_close for old in preserved_old):
            fixed_geoms.append(row)  # keep as-is
            continue


        # --- 4. Short & isolated? ------------------------------------------
        if geom.length < short_cut:
            continue  # DROP

        # --- 5. Extend ------------------------------------------------------
        new_pt = _extend_to_target(de_pt, prev_pt, gdf,sindex,idx, max_length=extend_max)
        # Build new LineString
        if start_de:
            new_coords = [(new_pt.x, new_pt.y),*coords]
        else:
            new_coords = [*coords,(new_pt.x, new_pt.y)]
        row['geometry'] = LineString(new_coords)
        fixed_geoms.append(row)

    # Combine kept originals + processed fixes
    gdf_out = GeoDataFrame(fixed_geoms, crs=gdf_edges.crs)
    return gpd.GeoDataFrame(pd.concat([keep_rows, gdf_out]), crs=project_crs)



In [210]:


# Step 1: Merge connected lines with same 'name' and 'highway'
merged_0 = merge_lines_by_attributes(gdf_fixed, attrs=['name', 'highway'])
merged = merged_0[merged_0.geometry.notnull()]

# Step 2: Build internal intersection points efficiently
internal_points = extract_internal_intersections_spatial_index(list(merged.geometry))

# Step 3: Split merged lines by these points
mid_result = split_lines_by_intersections_fast(merged, internal_points)


Extracting intersections: 100%|██████████| 6751/6751 [00:03<00:00, 1936.25it/s]


In [212]:

# preserved_* are sets of shapely Points
preserved_deadends_old = {
    pt for pt, val in extract_deadend_points(first_results).items() if val == 1
}
preserved_deadends_new = {
    pt for pt, val in extract_deadend_points(mid_result).items() if val == 1
}

In [213]:
mid_result.reset_index().to_file(f'{test_folder}/mid_result.shp')

In [214]:
final_result2 = mid_result.copy()
clean_edges = clean_and_fix_deadends(
    final_result2,
    preserved_old=preserved_deadends_old,
    preserved_new=preserved_deadends_new
)
clean_edges.to_file(f'{test_folder}/clean_edges.shp')

Processing edges:   1%|          | 35/5512 [00:00<00:54, 100.92it/s]

85


UnboundLocalError: cannot access local variable 'other_edges' where it is not associated with a value

In [101]:

# Step 1: Merge connected lines with same 'name' and 'highway'
merged_0 = merge_lines_by_attributes(clean_edges, attrs=['name', 'highway'])
merged = merged_0[merged_0.geometry.notnull()]

# Step 2: Build internal intersection points efficiently
internal_points = extract_internal_intersections_spatial_index(list(merged.geometry))

# Step 3: Split merged lines by these points
final_result = split_lines_by_intersections_fast(merged, internal_points)
final_result['length'] = final_result.length
final_result.reset_index(drop=True).to_file(f'{data_folder}/network_final2.shp')


Extracting intersections: 100%|██████████| 6673/6673 [00:01<00:00, 6614.06it/s]


In [ ]:
## Test Area

In [178]:
internal_points

[<POINT (853168.596 5636692.757)>,
 <POINT (855190.622 5632499.563)>,
 <POINT (847850.651 5626551.187)>,
 <POINT (857011.255 5631848.446)>,
 <POINT (851545.25 5638246.098)>,
 <POINT (851541.985 5633063.345)>,
 <POINT (862664.927 5636254.644)>,
 <POINT (856574.796 5632883.311)>,
 <POINT (853409.358 5634130.298)>,
 <POINT (855188.389 5633176.392)>,
 <POINT (844149.725 5630716.263)>,
 <POINT (854759.158 5631626.471)>,
 <POINT (846435.058 5633547.373)>,
 <POINT (854254.637 5635135.955)>,
 <POINT (865398.678 5642218.185)>,
 <POINT (852964.514 5632659.655)>,
 <POINT (857669.616 5637208.681)>,
 <POINT (856901.808 5634142.451)>,
 <POINT (853494.18 5635231.448)>,
 <POINT (850622.259 5629478.938)>,
 <POINT (848762.66 5634154.087)>,
 <POINT (853079.625 5639043.056)>,
 <POINT (851423.863 5639971.118)>,
 <POINT (852104.125 5632648.636)>,
 <POINT (854319.577 5634632.888)>,
 <POINT (853051.029 5634273.915)>,
 <POINT (849498.553 5642043.474)>,
 <POINT (851439.081 5629352.236)>,
 <POINT (856948.528 563

In [215]:
GeoDataFrame(geometry=internal_points, crs=final_result.crs).reset_index().to_file(f'{test_folder}/internal_points.shp')

In [198]:
gdf = merged.copy()
intersections = internal_points
len(intersections)

25567

In [204]:
[pt.buffer(buffer_eps).boundary for pt in intersections]

[<LINESTRING (853168.596 5636692.757, 853168.596 5636692.757, 853168.596 5636...>,
 <LINESTRING (855190.622 5632499.563, 855190.622 5632499.563, 855190.622 5632...>,
 <LINESTRING (847850.651 5626551.187, 847850.651 5626551.187, 847850.651 5626...>,
 <LINESTRING (857011.255 5631848.446, 857011.255 5631848.446, 857011.255 5631...>,
 <LINESTRING (851545.25 5638246.098, 851545.25 5638246.098, 851545.25 5638246...>,
 <LINESTRING (851541.985 5633063.345, 851541.985 5633063.345, 851541.985 5633...>,
 <LINESTRING (862664.927 5636254.644, 862664.927 5636254.644, 862664.927 5636...>,
 <LINESTRING (856574.796 5632883.311, 856574.796 5632883.311, 856574.796 5632...>,
 <LINESTRING (853409.358 5634130.298, 853409.358 5634130.298, 853409.358 5634...>,
 <LINESTRING (855188.389 5633176.392, 855188.389 5633176.392, 855188.389 5633...>,
 <LINESTRING (844149.725 5630716.263, 844149.725 5630716.263, 844149.725 5630...>,
 <LINESTRING (854759.158 5631626.471, 854759.158 5631626.471, 854759.158 5631...>,
 <LI

In [200]:
buffer_eps=1e-6
# not intersections
splitters = unary_union([pt.buffer(buffer_eps).boundary for pt in intersections])
len(splitters)

TypeError: object of type 'MultiLineString' has no len()

In [208]:
print(list(splitters.geoms)[0])

LINESTRING (847582.7852983978 5626389.84495878, 847582.7852961798 5626389.844953543)


In [197]:
# Convert to list of LineStrings if needed
if splitters.geom_type == 'LineString':
    lines = [splitters]
elif splitters.geom_type == 'MultiLineString':
    lines = list(splitters.geoms)
else:
    lines = []

# Create GeoDataFrame
gpd.GeoDataFrame(geometry=lines, crs=project_crs).to_file(f'{test_folder}/splitters.shp')

In [ ]:
prepared_splitters = prep(splitters)

In [199]:
row = gdf.loc[40]
geom = row.geometry
prepared_splitters.intersects(geom)


False

In [173]:
def split_lines_by_intersections_fast(gdf, intersections, buffer_eps=1e-6):
    """
    Splits line geometries at intersection points using a buffered boundary union.

    Parameters:
        gdf (GeoDataFrame): Input GeoDataFrame containing line geometries.
        intersections (list of Point): Points where lines should be split.
        buffer_eps (float): Buffer radius to create boundaries around each intersection point for splitting.

    Returns:
        GeoDataFrame: New GeoDataFrame with split line segments, preserving 'name' and 'highway' attributes.
    """
    if not intersections:
        return gdf.copy()

    # Build splitting geometry
    splitters = unary_union([pt.buffer(buffer_eps).boundary for pt in intersections])
    prepared_splitters = prep(splitters)

    records = []
    for row in gdf.itertuples(index=False):
        geom = row.geometry

        if prepared_splitters.intersects(geom):
            try:
                split_parts = split(geom, splitters)
                for part in split_parts.geoms:
                    if part.length > 0:
                        records.append({
                            'name': getattr(row, 'name', None),
                            'highway': getattr(row, 'highway', None),
                            'geometry': part
                        })
            except Exception:
                # Fallback in case of split failure
                records.append({
                    'name': getattr(row, 'name', None),
                    'highway': getattr(row, 'highway', None),
                    'geometry': geom
                })
        else:
            records.append({
                'name': getattr(row, 'name', None),
                'highway': getattr(row, 'highway', None),
                'geometry': geom
            })

    temp_geo = gpd.GeoDataFrame(records, crs=gdf.crs)
    gdf = temp_geo[temp_geo.length >1e-4]
    gdf['geometry'] = gdf['geometry'].apply(
    lambda geom: LineString([(round(x, 3), round(y, 3)) for x, y in geom.coords])
)
    return gdf

True

In [ ]:
if inter.geom_type == 'Point':
    all_points.add(inter)
elif inter.geom_type=='LineString':
    Point(inter.coords[0])
elif inter.geom_type.startswith('Multi') or inter.geom_type == 'GeometryCollection':
    for g in inter.geoms:
        if g.geom_type == 'Point':
            all_points.add(g)
        elif g.geom_type == 'LineString':
            all_points.add(Point(g.coords[0]))

In [ ]:

def extract_internal_intersections_spatial_index(geoms):
    """
    Extracts internal intersection points between geometries using spatial indexing (STRtree).

    Parameters:
        geoms (list of BaseGeometry): List of Shapely geometries (e.g., LineStrings).

    Returns:
        list of Point: Intersection points where geometries overlap internally.
    """
    geoms = [g for g in geoms if isinstance(g, BaseGeometry) and not g.is_empty]
    tree = STRtree(geoms)
    all_points = set()

    for idx, geom in enumerate(tqdm(geoms, total=len(geoms), desc="Extracting intersections")):
        candidate_idxs = tree.query(geom)
        candidates = [geoms[i] for i in candidate_idxs]

        for other in candidates:
            if other is geom or not geom.intersects(other):
                continue

            inter = geom.intersection(other)
            if inter.is_empty:
                continue

            if inter.geom_type == 'Point':
                all_points.add(inter)
            elif inter.geom_type.startswith('Multi') or inter.geom_type == 'GeometryCollection':
                all_points.update(g for g in inter.geoms if g.geom_type == 'Point')

    return list(all_points)

In [117]:

# --- 1. Annotate dead-end status ----------------------------------------
def _is_deadend(pt):
    return pt in preserved_deadends_new

gdf = mid_result.copy()
gdf["start_deadend"] = gdf.geometry.apply(lambda g: _is_deadend(g.coords[0]))
gdf["end_deadend"]   = gdf.geometry.apply(lambda g: _is_deadend(g.coords[-1]))
gdf["has_deadend"]   = gdf["start_deadend"] | gdf["end_deadend"]
gdf

,name,highway,geometry,start_deadend,end_deadend,has_deadend
1,11,path,"LINESTRING (854914.954 5625472.999, 854917.67 ...",False,False,False
3,11,path,"LINESTRING (857362.836 5626586.283, 857369.549...",True,True,True
5,11,path,"LINESTRING (857888.02 5626761.57, 857902.213 5...",False,False,False
7,18,path,"LINESTRING (860051.647 5627564.948, 860039.124...",True,False,True
10,18,path,"LINESTRING (860173.665 5627099.066, 860141.427...",False,True,True
...,...,...,...,...,...,...
39294,cavalcone A,footway,"LINESTRING (853160.753 5632645.818, 853160.753...",False,True,True
39296,strada alle sei ville,residential,"LINESTRING (858113.018 5631216.957, 858123.16 ...",False,True,True
39298,strada costalunga,residential,"LINESTRING (856768.157 5623617.085, 856771.006...",False,True,True
39300,via Edoardo Perroncito,service,"LINESTRING (846757.429 5633375.691, 846856.748...",False,True,True


In [118]:

# Split: rows with / without dead-ends
keep_rows = gdf[~gdf.has_deadend].copy()
fix_rows  = gdf[gdf.has_deadend].copy()

fixed_geoms = []

In [119]:
idx = 179
row= fix_rows.loc[idx]
row

name                                               Corso Adriatico
highway                                                   tertiary
geometry         LINESTRING (846805.407 5631751.103, 846829.987...
start_deadend                                                False
end_deadend                                                   True
has_deadend                                                   True
Name: 179, dtype: object

In [93]:
buffer_close  =20
short_cut  = 50
extend_max =  100
preserved_old  = preserved_deadends_old
geom = row.geometry
coords = list(geom.coords)
start_pt, end_pt = Point(coords[0]), Point(coords[-1])
start_de, end_de = row.start_deadend, row.end_deadend
# start_de and end_de
# start_de
# de_pt, prev_pt = start_pt, Point(coords[1])
de_pt, prev_pt = end_pt, Point(coords[-2])
any(de_pt.distance(Point(old)) < buffer_close for old in preserved_old)
geom.length < short_cut
new_pt = _extend_to_target(de_pt, prev_pt, gdf,sindex, max_length=extend_max)
if start_de:
    new_coords = [(new_pt.x, new_pt.y),*coords]
else:
    new_coords = [*coords,(new_pt.x, new_pt.y)]
new_coords

[(853880.997, 5633366.217),
 (853872.236, 5633370.93),
 (853769.933, 5633421.405),
 (853765.558, 5633423.407),
 (853754.916, 5633428.263),
 (853735.5411111057, 5633437.103862666)]

[(853699.0607100716, 5633360.965149304),
 (853717.034, 5633351.746),
 (853732.34, 5633343.895),
 (853832.806, 5633294.429),
 (853842.725, 5633289.621)]

In [17]:

def clean_and_fix_deadends(
        gdf_edges: gpd.GeoDataFrame,
        preserved_old: set,
        preserved_new: set,
        *,
        buffer_close: float = 20,
        short_cut: float = 50,
        extend_max: float = 100
) -> gpd.GeoDataFrame:
    """
    • Adds columns: start_deadend, end_deadend, has_deadend
    • Drops edges with dead-ends at *both* ends
    • For one-sided dead-ends:
        – If the dead-end is within `buffer_close` of any preserved_old → keep
        – Else, if length < `short_cut` → drop
        – Else → extend until it meets another edge (or nearest edge if none
          within `extend_max`)
    """
    # --- 1. Annotate dead-end status ----------------------------------------
    def _is_deadend(pt):
        return pt in preserved_new

    gdf = gdf_edges.copy()
    sindex = gdf.sindex
    gdf["start_deadend"] = gdf.geometry.apply(lambda g: _is_deadend(g.coords[0]))
    gdf["end_deadend"]   = gdf.geometry.apply(lambda g: _is_deadend(g.coords[-1]))
    gdf["has_deadend"]   = gdf["start_deadend"] | gdf["end_deadend"]

    # Split: rows with / without dead-ends
    keep_rows = gdf[~gdf.has_deadend].copy()
    fix_rows  = gdf[gdf.has_deadend].copy()

    fixed_geoms = []

    for idx, row in tqdm(fix_rows.iterrows(), total=len(fix_rows), desc="Processing edges"):
        geom = row.geometry
        coords = list(geom.coords)
        start_pt, end_pt = Point(coords[0]), Point(coords[-1])
        start_de, end_de = row.start_deadend, row.end_deadend

        # --- 2. Remove if dead-end at both ends -----------------------------
        if start_de and end_de:
            continue  # DROP edge entirely

        # Identify the dead-end & its previous vertex
        if start_de:
            de_pt, prev_pt = start_pt, Point(coords[1])
        else:
            de_pt, prev_pt = end_pt, Point(coords[-2])

        # --- 3. Close to preserved_old? -------------------------------------
        if any(de_pt.distance(Point(old)) < buffer_close for old in preserved_old):
            fixed_geoms.append(row)  # keep as-is
            continue


        # --- 4. Short & isolated? ------------------------------------------
        if geom.length < short_cut:
            continue  # DROP

        # --- 5. Extend ------------------------------------------------------
        new_pt = _extend_to_target(de_pt, prev_pt, gdf,sindex, max_length=extend_max)

        # Build new LineString
        if start_de:
            new_coords = [(new_pt.x, new_pt.y),*coords]
        else:
            new_coords = [*coords,(new_pt.x, new_pt.y)]
        row['geometry'] = LineString(new_coords)
        fixed_geoms.append(row)

    # Combine kept originals + processed fixes
    gdf_out = GeoDataFrame(fixed_geoms, crs=gdf_edges.crs)
    return gpd.GeoDataFrame(pd.concat([keep_rows, gdf_out], ignore_index=True), crs=project_crs)

[(853997.904, 5633456.685),
 (854022.061, 5633506.719),
 (854014.669, 5633522.389)]

In [83]:
de_pt = pt_deadend.coords[0]
prev_pt = pt_prev.coords[0]
vec = np.array(pt_deadend.coords[0]) - np.array(pt_prev.coords[0])
norm = np.linalg.norm(vec)

In [84]:
de_pt

(854014.669, 5633522.389)

In [ ]:

def _extend_to_target(pt_deadend: Point,
                      pt_prev: Point,
                      gdf_all: gpd.GeoDataFrame,
                      sindex,
                      max_length: float = 100):
    """
    Shoot a ray from the dead-end point in the direction of (pt_prev → pt_deadend)
    up to `max_length` and return a new point where it first meets another edge.
    If nothing is hit, return the projection of the dead-end onto the nearest edge.
    """
    # Direction vector (unit)
    vec = np.array(pt_deadend.coords[0]) - np.array(pt_prev.coords[0])
    norm = np.linalg.norm(vec)
    if norm == 0:         # safety
        return pt_deadend
    vec_unit = vec / norm

    ray_end = Point(pt_deadend.x + vec_unit[0] * max_length,
                    pt_deadend.y + vec_unit[1] * max_length)
    ray = LineString([pt_deadend, ray_end])

    # Spatial index for speed
    hits = list(sindex.intersection(ray.bounds))
    candidates0 = gdf_all.iloc[hits]
    candidates = candidates0[candidates0.index != idx].geometry
    # First intersection (if any)
    intersection_pts = [ray.intersection(g) for g in candidates if ray.intersects(g)]
    intersection_pts = [pt for pt in intersection_pts if not pt.is_empty]

    if intersection_pts:
        # Pick the closest intersection to the dead-end
        dists = [pt_deadend.distance(pt) for pt in intersection_pts]
        return intersection_pts[int(np.argmin(dists))]
    else:
        # Fall-back: project to closest edge (excluding the current ray)
        other_edges = gdf_all[~gdf_all.geometry.equals(pt_deadend)]
        nearest_geom = other_edges.geometry.unary_union
        proj = nearest_points(pt_deadend, nearest_geom)[1]
        return proj

In [ ]:
# Test Area - END